In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
sys.path.append("../")
import data_loading as dl
import os
from microfit import run_plotter as rp
from microfit import histogram as hist
import matplotlib.pyplot as plt
from microfit import variable_definitions as vdef
from microfit import selections

#### Make the same set of plots for each selection and each run ####

In [3]:
RUN_COMBOS_vv = [["1"],["2"],["3"],["4b"],["4c"],["4d"],["5"]]

In [4]:
SELECTION_v = ["SG_1MUNP"]
PRESELECTION_v = ["NUMU"]
VARIABLE_v = [vdef.tki_reco_variables_1mu1p]
DATASET="muon_sideband"

for run_combo in RUN_COMBOS_vv:
    
    # Make a directory to store the plots
    runcombo_str=""
    for i_r in range(0,len(run_combo)):
        runcombo_str = runcombo_str + run_combo[i_r]
    os.system("mkdir -p Plots/run_"+runcombo_str)
    
    print("Making plots for runs",run_combo)
    
    rundata, mc_weights, data_pot = dl.load_runs(
        run_combo,
        data=DATASET,
        truth_filtered_sets=["nue","drt"],
        loadpi0variables=False,
        loadshowervariables=False,
        loadrecoveryvars=False,
        loadsystematics=True,
        load_lee=False,
        blinded=False,
        enable_cache=True,
        cache_dir="/uboone/data/users/cthorpe/cached_dataframes/",
        numupresel=True,
        loadnumuvariables=True,
        use_bdt=False,
        load_numu_tki=True
    ) 
        
    for i in range(0,len(SELECTION_v)):
        selection = SELECTION_v[i]
        preselection = PRESELECTION_v[i]
        variables = VARIABLE_v[i]
        
        for binning_def in variables:
            
            # some binning definitions have more than 4 elements,
            # we ignore the last ones for now
            binning = hist.Binning.from_config(*binning_def[:4])
            signal_generator = hist.RunHistGenerator(
                rundata,
                binning,
                data_pot=data_pot,
                selection=selection,
                preselection=preselection,
                sideband_generator=None,
                uncertainty_defaults=None,
            )
            plotter = rp.RunHistPlotter(signal_generator)
            axes = plotter.plot(
                category_column="paper_category",
                include_multisim_errors=False,
                add_ext_error_floor=False,
                show_data_mc_ratio=True,
            )

            # Form a unique name for each plot
            pltname = "Plots/run_" + runcombo_str + "/" + DATASET + "_" + binning_def[0] + "_run" + runcombo_str + "_" + selection + ".pdf"
            print(pltname)
            print(type(plt))
            plt.savefig(pltname)
            
            #plt.show()
            
            
    del rundata
    del mc_weights
    del data_pot


Making plots for runs ['1']
Using data_paths.yml to locate ntuple file
get_rundict: run_number= 1
Loading ntuple file /uboone/data/users/cthorpe/PELEE_2023_Samples/run1/numupresel/NuMuSideband_bnb_beam_on_peleeTuple_uboone_v08_00_00_70_run1_C1.root


/uboone/app/users/cthorpe/Mambaforge/envs/python3LEE/lib/python3.7/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: overflow encountered in power
  result = getattr(ufunc, method)(*inputs, **kwargs)


SystemError: <built-in function is_scalar> returned a result with an error set

#### Same as Above, but Different Variables/Data Loading for the numu channels #####